# sources:

## audio preparation:
* https://www.tensorflow.org/tutorials/audio/simple_audio
* https://www.tensorflow.org/io/tutorials/audio

## siamese network:
* https://github.com/hlamba28/One-Shot-Learning-with-Siamese-Networks

## misc:
* https://gitlab.tu-berlin.de/dl4aed/dl4aed-lectures/blob/master/04-audio-preprocessing.ipynb

In [ ]:
%%bash
# install and update required packages
python3 -m pip install --upgrade pip -q
python3 -m pip install -r requirements.txt -q

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
''' Small hack to restart kernel after netbook ran to free GPU memory '''
## restart jupyter kernel to free all memory in notebook
from IPython.display import display_html
def restartkernel():
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)

In [ ]:
''' Some imports '''
import tensorflow as tf
assert tf.__version__ >= "2.4.0"
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
''' Allow typical dynamic GPU memory allocation and read config'''
from utils import load_config, save_config, allow_growth, update
allow_growth()

# read the config file
# it includes more or less all hyperparameter used in the model and preprocessing/training step
config = load_config(verbose=0)

# some other configuration (*.py)
import configuration

In [ ]:
''' Parsing CLI arguments and overwriting config '''
from argument_parser import parse_arguments
parsed_arguments, _ = parse_arguments()

In [ ]:
# overwrite config
config = update(config, configuration.config)
config = update(config, parsed_arguments.__dict__)

In [ ]:
import os
os.environ["WANDB_PROJECT"] = "resnet-experiments"
os.environ["WANDB_ENTITY"] = "dl4aed"
os.environ["WANDB_API_KEY"] = "***"
os.environ["WANDB_MODE"] = "dryrun" #or "run"
os.environ["WANDB_NOTEBOOK_NAME"] = "model"

In [ ]:
from wandb_utils import WandbWrapper
wandb_wrapper = WandbWrapper(config)
config = wandb_wrapper.get_config()

In [ ]:
# extract parameter classes
data_parameter = config["data_parameter"]
model_parameter = config["model_parameter"]
training_parameter = config["training_parameter"]

In [ ]:
''' Some small amount of reproducibility '''
tf.random.set_seed(training_parameter["seed"])
np.random.seed(training_parameter["seed"])

In [ ]:
''' Create model and save config '''
from network import Network
network = Network(model_parameter, training_parameter)
network.compile()
#network.save()
# save config into model path
#save_config(data_parameter, model_parameter, training_parameter, network)

In [ ]:
config = {}
config['data_root'] = 'gtzan'
config['noise_path'] = '/media/datasets/fsdkaggle2018/FSDKaggle2018.meta/test_post_competition_scoring_clips.csv'
config['sample_rate'] = 22050
config['nfft'] = 2048
config['window'] = 2048
config['stride'] = 1024
config['mels'] = 64
config['fmin_mels'] = 0
config['fmax_mels'] = 8000
config['time_mask'] = 10
config['freq_mask'] = 10
config['noise_threshold'] = 1 # add noise to only 0.3
config['beta'] = 0.5 # noise strength when mixing mel spectrograms
config['SNR'] = 1.
config['noise_root'] = '/media/datasets/fsdkaggle2018/FSDKaggle2018.audio_test/'
config['shuffle_buffer_size'] = 1000
config['batch_size'] = 64

from preprocessor import Preprocessor
prep = Preprocessor(config=config)
prep.create_logger()

prep.set_config({'fade': 10000,
              'epsilon': 0.1,
              'roll_val': 1024,
              'top_db': 80,
              'shift_val': 3,
              'bins_per_octave': 12,
              'param_db': 10,
              'train_size': 0.7,
              'val_size': 0.2,
              'test_size': 0.1,
              'noisy_samples': 5,
              "common_divider": 64,})

prep.load_data(data_dir="/media/datasets/tfds/")
for mode in prep.available_modi:
    prep.offline_preprocessing(mode)
prep.save_mels()

In [ ]:
prep.load_mels()

In [ ]:
prep.launch_trainval_pipeline(mode="train")
prep.launch_trainval_pipeline(mode="val")
prep.launch_test_pipeline()

In [ ]:
''' Training procedure '''
history = network.fit(prep.train_ds, validation_data=prep.val_ds, epochs=training_parameter["epochs"],
                   initial_epoch=network.epoch, callbacks=network.callbacks + [wandb_wrapper.get_callback(save_model=False)])

In [ ]:
import pandas as pd
pd.DataFrame.from_dict(history.history).plot()

In [ ]:
''' Evaluation procedure '''
import wandb

collect_embeddings = []
for elem in prep.val_ds:
    embedding = network.predict_embedding_on_batch(elem[0])
    collect_embeddings += [embedding]
embeddings = tf.concat(collect_embeddings, axis=0)
print(embeddings.shape)

In [ ]:
x = tf.reshape(embeddings,(-1,embeddings.shape[-1]))
from sklearn.manifold import TSNE
X_embedded = TSNE(n_components=2).fit_transform(x)

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.ticker import NullFormatter

# https://stackoverflow.com/a/57222323
from itertools import cycle
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = cycle(prop_cycle.by_key()['color'])

# Create figure
fig = plt.figure(figsize=(8, 8))
# Add scatter plot
ax = fig.add_subplot(111)
ax.scatter(X_embedded[:, 0], X_embedded[:, 1], alpha=0.05)
for i in range(10):
    sub_images = 10
    ax.scatter(X_embedded[i*sub_images:(i+1)*sub_images, 0], X_embedded[i*sub_images:(i+1)*sub_images, 1], alpha=0.5, c=next(colors))
ax.xaxis.set_major_formatter(NullFormatter())
ax.yaxis.set_major_formatter(NullFormatter())
ax.axis('tight')
wandb.log({"embedding": wandb.Image(plt)})

In [ ]:
from matplotlib import pyplot as plt
for elem in prep.val_ds:
    prediction = network.predict_on_batch(elem[0])
    for (x, y) in list(zip(elem[0], prediction))[:10]:
        x = x[...,0].numpy().astype(np.float32).T
        y = y[...,0].astype(np.float32).T
        wandb_wrapper.post_plt_image(x, y, title="Images", tag="side-by-side-images")
        wandb_wrapper.post_plt_histogram(x, y, title="Histogram", tag="overlay-histogram", alpha=0.35, bins=50)
    break

In [ ]:
# restart kernel to free GPU mem
#restartkernel()

In [ ]:
import numpy as np
collect_embeddings = []
collect_labels = []
for elem in prep.test_ds:
    for (x, y) in zip(elem[0], elem[1]):
        prediction = network.predict_embedding_on_batch(x[np.newaxis])
        collect_embeddings += [prediction]
        collect_labels += [y]

In [ ]:
embeddings = tf.concat(collect_embeddings, axis=0)
labels = np.concatenate(collect_labels, axis=0)
x = tf.reshape(embeddings,(-1,embeddings.shape[-1]))
from sklearn.manifold import TSNE
X_embedded = TSNE(n_components=2).fit_transform(x)

In [ ]:
num_classes = 10
collect_colors_markers = {}

from itertools import cycle
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = cycle(prop_cycle.by_key()['color'])
#markers = cycle(('x', ',', '+', '.', 'o', '*'))
markers = cycle(('o', ','))

for i in range(num_classes):
    collect_colors_markers[i] = (next(colors), next(markers))

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.ticker import NullFormatter

# Create figure
fig = plt.figure(figsize=(8, 8))
# Add scatter plot
ax = fig.add_subplot(111)

for embedding, label in zip(X_embedded, labels):
    ax.scatter(embedding[0], embedding[1], alpha=0.5, c=collect_colors_markers[label][0], marker=collect_colors_markers[label][1])

ax.xaxis.set_major_formatter(NullFormatter())
ax.yaxis.set_major_formatter(NullFormatter())
ax.axis('tight')
wandb.log({"test embedding - label colored": wandb.Image(plt)})